# Import

In [1]:
!git clone -b deep-learning-models https://github.com/MohamadElnomrossie/Omdena-seniment-analysis/

fatal: destination path 'Omdena-seniment-analysis' already exists and is not an empty directory.


In [2]:
%cd Omdena-seniment-analysis/

/content/Omdena-seniment-analysis


In [3]:
# !git clone -b Tokenization https://github.com/OmdenaAI/Arabic-Chapter/

# Main

In [4]:
import os
import json

import numpy as np
import pandas as pd

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from sklearn.model_selection import StratifiedKFold
import tensorflow as tf

from Sentiment import SentimentAnalysis
from utils import helper, preprocess
# from utils.config import config

# from ArabicChapter.Tokenizer.tokenizer import tokenization

### Analysis

In [5]:
import pandas as pd
data = pd.read_csv("Datasets/Final_Dataset/Dataset/train.csv")
print(data.columns)
data.head(10)

Index(['Unnamed: 0', 'Text', 'Class_camel', 'cleaned_text', 'mentions',
       'word_count', 'contain_emoji', 'demoji_text', 'URLS'],
      dtype='object')


,Unnamed: 0,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS
0,0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,neutral,حقوق المراه,[],2,True,:green_heart::green_heart::green_heart:,['https://t.co/Mzf90Ta5g1']
1,2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (...,neutral,لجنه التنميه بشبرا ما زال التسجيل مستمر في دور...,"['saud', 'Shubratanmyeh']",15,True,:sparkles:,['https://t.co/c2NXzNCdLU']
2,5,RT @fatemaar7: انا كامرأة يوجعني كل حزن نساء ا...,negative,انا كامراه يوجعني كل حزن نساء العالم سواء تعنف...,['fatemaar7'],50,False,NaN,['https://t.co/c494qBVPx4']
3,7,@AliFCD @ShamsanM @AlyemenNor @space_dist @m2r...,negative,غسق وسبيس وحلوه الحلوات هم اشخاص لهم هويتهم وك...,"['AliFCD', 'ShamsanM', 'AlyemenNor', 'space', ...",32,False,NaN,[]
4,9,RT @Dresraamohammed: السؤال بقى للناس الي شغال...,negative,السؤال بقي للناس الي شغاله في حقوق الانسان حقو...,['Dresraamohammed'],24,True,:person_facepalming_light_skin_tone::female_si...,[]
5,12,إذا كانت مسألة حقوق المرأة أمرا سخيفاً جداً ، ...,negative,اذا كانت مساله حقوق المراه امرا سخيفا جدا فالل...,[],18,False,NaN,[]
6,13,@naifco كل هذه الحقوق او المزعوم انها حقوق عبا...,negative,كل هذه الحقوق او المزعوم انها حقوق عباره عن صف...,['naifco'],16,False,NaN,[]
7,14,متى تعرف ان المرأة ما عندها حقوق؟\n يوم يطبعون...,neutral,متي تعرف ان المراه ما عندها حقوق يوم يطبعون له...,[],13,True,:backhand_index_pointing_down_medium-light_ski...,['https://t.co/7a9H0KQTqH']
8,16,RT @miss_wahaj: @RmNxb المسيار زي زواج المتعه ...,negative,المسيار زي زواج المتعه عند الشيعه نعايب عليهم ...,"['miss', 'RmNxb']",13,False,NaN,[]
9,18,RT @Gh08944301: @Eqbal_Darandari لمتى واحنا بل...,negative,لمتي واحنا بلا حقوق لانقدر نجدد جوازات سفرنا و...,"['Gh08944301', 'Eqbal']",50,False,NaN,[]


In [6]:
print(data.cleaned_text.loc[45], data.Class_camel.loc[45])
print(data.cleaned_text.loc[0], data.Class_camel.loc[0])
print(data.cleaned_text.loc[35], data.Class_camel.loc[35])

لان تبني حقوق المراه من وجهه نظر علمانيه وتحرريه يضر بمجتمعي ويضر بالنساء والرجال علي حد سواء بس هم ليش مهتمين وحاشرين خشومهم negative
حقوق المراه neutral
تضامن مع ترف العسيري اختي المسلمه حجابك حشمتك حيائك هي هويتك الحقيقيه التي كرمك بها دينك العظيم حافظي عليها فهي من تمنحك الكرامه والحريه الحقيقيه لاتستمعي لمن يحاول طمس هويتك الاسلاميه تحت دعاوي مزيفه مثل حقوق المراه وبكائيات من تسمين انفسهن ب الناشطات والحقوقيات positive


In [7]:
data.Class_camel.value_counts()

positive    57096
negative    33702
neutral     20124
Name: Class_camel, dtype: int64

### Config

In [4]:
import pickle
with open("Datasets/stopWords.pkl", 'rb') as f:
    stop_words = list(pickle.load(f))
    stop_words = list(set(stop_words + ['و','في','من','بواسطة','أ','هو','و','في','سيكون','إلى','كان','كن','هو','ال','و','ما','ء','ه','س']))

config = {
    'vocab_size':60000,
    'maxlen':256,
    'embedding_vector':50,

    'method':'lstm', #other - simpleRNN, bidRNN, 1DConv, lstm
    'stop_words':stop_words,
    'punctuations':"""'!"-#$%&'()*+,«».؛،/:؟?@[\]^_`{|}~""",

    'epochs':50,
    'optim':'SGD', # other - adamax, adadelta, SGD, Adam, RMSprop
    'learning_rate':1e-1,

    'save_model_path':'models/',
    'save_weights_path':"models/",
    'train_data_path':"Datasets/Final_Dataset/Dataset/train.csv",
    'val_data_path':"Datasets/Final_Dataset/Dataset/val.csv",
    'test_data_path':"Datasets/Final_Dataset/Dataset/test.csv",
}


### Fine Tuning

In [15]:
train_data = pd.read_csv(config['train_data_path'])
val_data = pd.read_csv(config['val_data_path'])
test_data = pd.read_csv(config['test_data_path'])

train_data = train_data.dropna().reset_index(drop=True)
val_data = val_data.dropna().reset_index(drop=True)
test_data = test_data.dropna().reset_index(drop=True)

train_text, train_label = train_data['cleaned_text'].values, train_data['Class_camel'].values
val_text, val_label = val_data['cleaned_text'].values, val_data['Class_camel'].values
test_text, test_label = test_data['cleaned_text'].values.copy(), test_data['Class_camel'].values.copy()


In [16]:
#Default Tokenization
sentiment = SentimentAnalysis(preprocess.tokenizer, vocab_size=config['vocab_size'], maxlen=config['maxlen'], embedding_vector=config['embedding_vector'], method=config['method'],)

train_text = sentiment.tokenize(train_text, punctuations=config['punctuations'], stop_words=config['stop_words'])
train_text, train_label, unique_words, word_dict = sentiment.vectorize(train_text, train_label)

val_text = sentiment.tokenize(val_text, punctuations=config['punctuations'], stop_words=config['stop_words'])
val_text, val_label, _, _ = sentiment.vectorize(val_text, val_label)

test_text = sentiment.tokenize(test_text, punctuations=config['punctuations'], stop_words=config['stop_words'])
test_text, test_label, _, _ = sentiment.vectorize(test_text, test_label)


100%|██████████| 15634/15634 [00:02<00:00, 6666.05it/s]


In [19]:
train_data.cleaned_text.iloc[0], train_data.Class_camel.iloc[0] 

(['حقوق', 'المراه'], 'neutral')

In [20]:
train_text[0], train_label[0]

(array([49220, 36270,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [21]:
model = sentiment.fit(train_text,
                    train_label,
                    validation_data=(val_text, val_label),
                    epochs=config['epochs'],
                    method=config['method'])
sentiment.evaluate(test_text, test_label, model, batch_size=32)

(29029, 256) (7267, 256) (29029, 3) (7267, 3)
Epoch 1/50
908/908 [==============================] - 32s 25ms/step - loss: 0.8863 - accuracy: 0.5604 - precision: 0.6677 - recall: 0.3212 - auc: 0.7673 - val_loss: 0.7561 - val_accuracy: 0.6623 - val_precision: 0.7291 - val_recall: 0.5504 - val_auc: 0.8444

Epoch 00001: val_loss improved from inf to 0.75613, saving model to models/lstm_model.h5
Epoch 2/50
908/908 [==============================] - 22s 24ms/step - loss: 0.6783 - accuracy: 0.7022 - precision: 0.7637 - recall: 0.5990 - auc: 0.8752 - val_loss: 0.6530 - val_accuracy: 0.7125 - val_precision: 0.7541 - val_recall: 0.6527 - val_auc: 0.8839

Epoch 00002: val_loss improved from 0.75613 to 0.65300, saving model to models/lstm_model.h5
Epoch 3/50
908/908 [==============================] - 22s 24ms/step - loss: 0.5796 - accuracy: 0.7565 - precision: 0.7990 - recall: 0.6928 - auc: 0.9101 - val_loss: 0.6070 - val_accuracy: 0.7350 - val_precision: 0.7725 - val_recall: 0.6845 - val_auc: 0.9

### Validation

In [32]:
print(f"\nText : {test_data['cleaned_text'].iloc[45]} Label : {test_data['Class_camel'].iloc[45]}")
sentiment.predict_([test_data['cleaned_text'].iloc[45]], model, batch_size=32, print_=True)

print(f"Text : {test_data['cleaned_text'].iloc[0]} Label : {test_data['Class_camel'].iloc[0]}")
sentiment.predict_([test_data['cleaned_text'].iloc[0]], model, batch_size=32, print_=True)

print(f"Text : {test_data['cleaned_text'].iloc[35]} Label : {test_data['Class_camel'].iloc[35]}")
sentiment.predict_([test_data['cleaned_text'].iloc[35]], model, batch_size=32, print_=True)

100%|██████████| 1/1 [00:00<00:00, 290.54it/s]


Text : سعوديات نطلب اسقاط الولايه سعوديات نطلب اسقاط الولايه هلو بابا سلمان طولتوا مو قال محمد بن سلمان ان الولايه راح تسقط قريب متي هالقريب تري راح عمرنا واحنا ننتظر اسقوطها بليز واعتقونا Label : negative
--------------------
Negative 0.9940369129180908
--------------------
Text : مبدائيا انا مع حقوق المراه وضد حقوق الرجل Label : neutral
--------------------
Neutral 0.5456753969192505
--------------------
Text : تضامن مع ترف العسيري فئه اسقاط الولايه القذره حثاله المجتمع الذين سمعوا عن الحريه فنادوا بخلع الحجاب والملابس ويعتبرونها حريهالشخص الحجاب عباده Label : negative
--------------------
Negative 0.9165098071098328
--------------------


### Inference

In [32]:
import pandas as pd
import tensorflow as tf
from Sentiment import SentimentAnalysis
from utils import helper, preprocess

data = pd.read_csv("data path")
data.head(10)

,Unnamed: 0,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS
0,0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,neutral,حقوق المراه,[],2,True,:green_heart::green_heart::green_heart:,['https://t.co/Mzf90Ta5g1']
1,2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (...,neutral,لجنه التنميه بشبرا ما زال التسجيل مستمر في دور...,"['saud', 'Shubratanmyeh']",15,True,:sparkles:,['https://t.co/c2NXzNCdLU']
2,5,RT @fatemaar7: انا كامرأة يوجعني كل حزن نساء ا...,negative,انا كامراه يوجعني كل حزن نساء العالم سواء تعنف...,['fatemaar7'],50,False,NaN,['https://t.co/c494qBVPx4']
3,7,@AliFCD @ShamsanM @AlyemenNor @space_dist @m2r...,negative,غسق وسبيس وحلوه الحلوات هم اشخاص لهم هويتهم وك...,"['AliFCD', 'ShamsanM', 'AlyemenNor', 'space', ...",32,False,NaN,[]
4,9,RT @Dresraamohammed: السؤال بقى للناس الي شغال...,negative,السؤال بقي للناس الي شغاله في حقوق الانسان حقو...,['Dresraamohammed'],24,True,:person_facepalming_light_skin_tone::female_si...,[]
5,12,إذا كانت مسألة حقوق المرأة أمرا سخيفاً جداً ، ...,negative,اذا كانت مساله حقوق المراه امرا سخيفا جدا فالل...,[],18,False,NaN,[]
6,13,@naifco كل هذه الحقوق او المزعوم انها حقوق عبا...,negative,كل هذه الحقوق او المزعوم انها حقوق عباره عن صف...,['naifco'],16,False,NaN,[]
7,14,متى تعرف ان المرأة ما عندها حقوق؟\n يوم يطبعون...,neutral,متي تعرف ان المراه ما عندها حقوق يوم يطبعون له...,[],13,True,:backhand_index_pointing_down_medium-light_ski...,['https://t.co/7a9H0KQTqH']
8,16,RT @miss_wahaj: @RmNxb المسيار زي زواج المتعه ...,negative,المسيار زي زواج المتعه عند الشيعه نعايب عليهم ...,"['miss', 'RmNxb']",13,False,NaN,[]
9,18,RT @Gh08944301: @Eqbal_Darandari لمتى واحنا بل...,negative,لمتي واحنا بلا حقوق لانقدر نجدد جوازات سفرنا و...,"['Gh08944301', 'Eqbal']",50,False,NaN,[]


In [33]:
data = data.dropna().reset_index(drop=True)
text = data["Text"].values

In [34]:
text[0:5]

array(['حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1',
       'RT @saud_talep: Retweeted لجنة التنمية بشبرا (@Shubratanmyeh):\n \n ما زال التسجيل مستمر في دورة حقوق المرأة بعد الطلاق ✨ #وعيك_يحميك... https://t.co/c2NXzNCdLU',
       'RT @Dresraamohammed: السؤال بقى للناس الي شغاله في #حقوق_الانسان #حقوق_المرأة \n يا ترى في قانون او عقوبه على الزوج الي بيمارس العنف ضد زوجته في الشارع ؟؟؟ 🤦🏻\u200d♀️👩🏻\u200d💼',
       'متى تعرف ان المرأة ما عندها حقوق؟\n يوم يطبعون لها ورقة مثل هذي 👇🏼👇🏼👇🏼 https://t.co/7a9H0KQTqH',
       '@Almajlliss اتوقع الي حاط الاستفتاء وده يخنق المرأه ويسجنها 😹😹 الحمدالله ديرتنا ديره حقوق للمرأه 🙏😋'],
      dtype=object)

In [35]:
model = tf.keras.models.load_model("models/lstm_model.h5")
preds = sentiment.predict_(text, model, batch_size=32, print_=False)
data["prediction"] = preds

100%|██████████| 29029/29029 [00:04<00:00, 5984.11it/s]


In [36]:
data.head()

,Unnamed: 0,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS,prediction
0,0,"[حقوق, المرأة, 💚💚💚, https, t, co]",neutral,حقوق المراه,[],2,True,:green_heart::green_heart::green_heart:,['https://t.co/Mzf90Ta5g1'],Negative
1,2,"[RT, saud, talep, Retweeted, لجنة, التنمية, بش...",neutral,لجنه التنميه بشبرا ما زال التسجيل مستمر في دور...,"['saud', 'Shubratanmyeh']",15,True,:sparkles:,['https://t.co/c2NXzNCdLU'],Negative
2,9,"[RT, Dresraamohammed, السؤال, بقى, للناس, الي,...",negative,السؤال بقي للناس الي شغاله في حقوق الانسان حقو...,['Dresraamohammed'],24,True,:person_facepalming_light_skin_tone::female_si...,[],Negative
3,14,"[متى, تعرف, ان, المرأة, عندها, حقوق, يوم, يطبع...",neutral,متي تعرف ان المراه ما عندها حقوق يوم يطبعون له...,[],13,True,:backhand_index_pointing_down_medium-light_ski...,['https://t.co/7a9H0KQTqH'],Negative
4,21,"[Almajlliss, اتوقع, الي, حاط, الاستفتاء, وده, ...",negative,اتوقع الي حاط الاستفتاء وده يخنق المراه ويسجنه...,['Almajlliss'],13,True,:cat_face_with_tears_of_joy::cat_face_with_tea...,[],Positive
